# Author imformation:
Fan Shengzhe, Shanghaijiaotong University, Shanghai, China  
Email: fanshengzhe@sjtu.edu.cn

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("drive/My Drive/crop_identification2/crop_identification")

# Install suitable tensorflow version
Notice: This script is based on tensorflow 2.x, and tensorflow 1.x will not be supported.

In [ ]:
import tensorflow as tf 
print(tf.__version__)
# if tf.__version__ != '2.8.2':
#   !pip install tensorflow==2.8.2

2.9.2


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18366238995071460134
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14415560704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4137842114555822142
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
try:
  import tensorflow_addons
except:
  !pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 30.5 MB/s 


# copy data to temporary folder

copy the data to the temporary folder in colab instead read the data from google drive may solve the problem which is extremely slow in the first epoch click the following URL for details: https://discuss.pytorch.org/t/drastically-slow-speed-at-first-epoch/12551/9

copying folder may be slow, zip the folder may solve this porblem 


In [ ]:
if not (os.path.exists('/content/data.rar') or os.path.exists('/content/data')):
  !cp data.rar /content/ 

In [ ]:
if not os.path.exists('/content/data'):
  !unrar x -inul /content/data.rar /content
  !rm -rf /content/data.rar

In [ ]:
DATA_ROOT_PATH = '/content/data'

# Import necessary modules

In [ ]:
import tensorflow as tf
import itertools
import datetime
import time
import json
import math
import io
from matplotlib import pyplot as plt
from functools import partial
from model_builder import *
from config import *
from utils import *

In [ ]:
MODEL_NAME = 'ghost_efficientnet_v2_s'
BATCH_SIZE = 32

# Training

In [ ]:
@tf.function
def train_step(input_img, label):
    with tf.GradientTape() as tape:
        pred = model(input_img, training=True)
        loss = loss_obj(label, pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer_obj.apply_gradients(zip(gradients, model.trainable_variables))
    metrics_obj.update_state(label, pred)
    return loss

@tf.function
def test_step(input_img, label):
    pred = model(input_img, training=False)
    loss = loss_obj(label, pred)
    metrics_obj.update_state(label, pred)
    return loss


def fit(train_ds, total_epochs, val_ds=None):
    current_epoch = 0
    best_metrics = 0
    best_epoch = 0

    # load ckpt here
    if os.path.exists(os.path.join(DAILY_CHECKPOINT_DIR, MODEL_NAME)):
        daily_ckpt.restore(tf.train.latest_checkpoint(os.path.join(DAILY_CHECKPOINT_DIR, MODEL_NAME)))
        print(f'INFO: ckpt found, restarted from epoch {epoch_recoder.numpy()}')
        print('-' * 80)
        current_epoch = epoch_recoder.numpy() + 1
        best_metrics = metrics_recoder.numpy()
        best_epoch = best_epoch_recoder.numpy()
    else:
        print('INFO: No ckpt found, start new training...')
        print('-' * 80)
    for _ in itertools.count():
        start_time = time.time()
        print(MODEL_NAME, '\t', "Epoch ", current_epoch)

        train_total_loss = 0
        batch_num = 1
        metrics_obj.reset_states()
        for n, (input_img, label) in train_ds.enumerate():
            loss = train_step(input_img, label)#, current_epoch)
            train_total_loss += loss
            batch_num = n + 1
            print('\rtrain', '\t', 'loss:', loss.numpy(), '\t', 
                'metrics:', metrics_obj.result().numpy(), '\t',
                'lr:', lr_schedule(optimizer_obj.iterations).numpy(), end='')
        metrics = metrics_obj.result()
        train_total_loss /= tf.cast(batch_num, tf.float32)
        with summary_writer.as_default():
            tf.summary.scalar('train_loss', train_total_loss, step=current_epoch)
            tf.summary.scalar('train_metrics', metrics, step=current_epoch)
            tf.summary.scalar('lr', lr_schedule(optimizer_obj.iterations),step=current_epoch)
        print('\rtrain', '\t', 'loss:', train_total_loss.numpy(), '\t', 
            'metrics:', metrics.numpy(), '\t',
            'lr:', lr_schedule(optimizer_obj.iterations).numpy())

        if val_ds is not None:
            val_total_loss = 0
            batch_num = 1
            metrics_obj.reset_states()
            for n, (input_img, label) in val_ds.enumerate():
                loss = test_step(input_img, label) #, current_epoch)
                val_total_loss += loss
                batch_num = n + 1
                print('\rval', '\t', 'loss:', loss.numpy(), '\t',
                  'metrics:', metrics_obj.result().numpy(), end='')
            val_total_loss /= tf.cast(batch_num, tf.float32)
            metrics = metrics_obj.result()

            if metrics > best_metrics:
                best_metrics = metrics
                best_epoch = current_epoch
                metrics_recoder.assign(best_metrics)
                epoch_recoder.assign(current_epoch)
                best_epoch_recoder.assign(best_epoch)
                best_ckpt_manager.save(checkpoint_number=best_epoch)

            with summary_writer.as_default():
                tf.summary.scalar('val_loss', val_total_loss, step=current_epoch)
                tf.summary.scalar('val_metrics', metrics, step=current_epoch)
            print('\rval', '\t', 'loss:', val_total_loss.numpy(), '\t',
               'metrics:', metrics.numpy(), '\t',
               'current_best_epoch:', best_epoch)


        if (current_epoch + 1) % 1 == 0:
            # save the model every epoches
            epoch_recoder.assign(current_epoch)
            daily_ckpt_manager.save(checkpoint_number=current_epoch)
        end_time = time.time()
        print('Time taken for epoch {} is {} sec\n'.format(current_epoch, end_time - start_time))
        print('-' * 80)
        with summary_writer.as_default():
            tf.summary.scalar('time taken', end_time - start_time, step=current_epoch)

        current_epoch += 1
        if current_epoch >= EPOCHS:
            print('training done')
            break

    # save the model when all epochs have been done
    daily_ckpt_manager.save(checkpoint_number=EPOCHS)


if __name__ == "__main__":
    # Creat class dict
    if not os.path.exists(os.path.join(CLASS_DICT_DIR, "class_dict.json")):
      if not os.path.exists(CLASS_DICT_DIR):
        os.mkdir(CLASS_DICT_DIR)
      cls_dict, rev_cls_dict = make_class_dict(DATA_ROOT_PATH, TRAIN_DIR_NAME, TEST_DIR_NAME, VAL_DIR_NAME)
      with open(f"{CLASS_DICT_DIR}/class_dict.json", "w") as jfile:
        json.dump((cls_dict, rev_cls_dict), jfile)
      print("WARN: no class_dict found, created new one. The model must be trained from scratch")
    else:
      jfile = open(f"{CLASS_DICT_DIR}/class_dict.json", "r")
      cls_dict, rev_cls_dict = json.load(jfile)
      print("INFO: class_dict found and reused")
    
    # Define model
    model = get_model(MODEL_NAME, IMAGE_H, IMAGE_W, 3, len(cls_dict))
    # Plot NN Arch
    # model.build((100, IMAGE_H, IMAGE_W, 3))
    # model.summary()
    # tf.keras.utils.plot_model(model)

    # Creat data pipeline
    train_image_path_list, train_label = get_img_path_list(DATA_ROOT_PATH, TRAIN_DIR_NAME, class_dict=cls_dict, one_hot=True)
    train_dataset = tf.data.Dataset.from_tensor_slices((train_image_path_list, train_label))
    train_dataset = train_dataset.shuffle(buffer_size=100)
    tsfm_train = partial(transform_train, height=IMAGE_H, width=IMAGE_W)
    train_dataset = train_dataset.map(tsfm_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    val_image_path_list, val_label = get_img_path_list(DATA_ROOT_PATH, VAL_DIR_NAME, class_dict=cls_dict, one_hot=True)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_image_path_list, val_label))
    tsfm_val = partial(transform_test, height=IMAGE_H, width=IMAGE_W)
    val_dataset = val_dataset.map(tsfm_val, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    
    # Show example
    # for inp, label in train_dataset.take(10):
    #     plt.imshow(inp[0] * 0.5 + 0.5)
    #     plt.title(rev_cls_dict[int(tf.argmax(label[0], axis=0).numpy())])
    #     plt.axis('off')
    #     plt.tight_layout()
    #     plt.show()
    
    # Loss, metrices, lr schedule and optimizer
    loss_obj = tf.keras.losses.CategoricalCrossentropy()
    metrics_obj = tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')
    steps_per_epoch = math.ceil(len(train_image_path_list)/BATCH_SIZE)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(LEARNING_RATE, decay_steps=20*steps_per_epoch, decay_rate=0.1)
    # lr_schedule = tf.keras.experimental.CosineDecay(LEARNING_RATE, DECAY_STEPS, alpha=0.01)
    # DECAY_STEPS = 2*steps_per_epoch
    # lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(LEARNING_RATE, first_decay_steps=DECAY_STEPS, \
    #                                   t_mul=2.0, m_mul=0.75, alpha=0.001)

    optimizer_obj = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    # Recoder and checkpoints
    epoch_recoder = tf.Variable(0)
    metrics_recoder = tf.Variable(0.)
    best_epoch_recoder = tf.Variable(0)

    daily_ckpt = tf.train.Checkpoint(optimizer=optimizer_obj,
                      model=model,
                      current_epoch=epoch_recoder,
                      current_best_metrics=metrics_recoder,
                      current_best_epoch=best_epoch_recoder)
    daily_ckpt_manager = tf.train.CheckpointManager(daily_ckpt, os.path.join(DAILY_CHECKPOINT_DIR, MODEL_NAME), 3)

    best_ckpt = tf.train.Checkpoint(optimizer=optimizer_obj,
                      model=model,
                      current_epoch=epoch_recoder,
                      current_best_metrics=metrics_recoder,
                      current_best_epoch=best_epoch_recoder)
    best_ckpt_manager = tf.train.CheckpointManager(best_ckpt, os.path.join(BEST_CHECKPOINT_DIR, MODEL_NAME), 3)

    # Tensorboard summary
    summary_writer = \
    tf.summary.create_file_writer(os.path.join(LOG_DIR, MODEL_NAME, f"bs{BATCH_SIZE}_lr{LEARNING_RATE}_ds{DECAY_STEPS}",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
    # Training
    fit(train_dataset, EPOCHS, val_dataset)


INFO: class_dict found and reused
INFO: ckpt found, restarted from epoch 6
--------------------------------------------------------------------------------
ghost_efficientnet_v2_s 	 Epoch  7
train 	 loss: 1.4579751 	 metrics: 0.60475755 	 lr: 0.00039810716
val 	 loss: 1.1901783 	 metrics: 0.6746927 	 current_best_epoch: 7
Time taken for epoch 7 is 2884.9259119033813 sec

--------------------------------------------------------------------------------
ghost_efficientnet_v2_s 	 Epoch  8
train 	 loss: 1.3352083 	 metrics: 0.63476807 	 lr: 0.00035481344
val 	 loss: 1.1565617 	 metrics: 0.68457735 	 current_best_epoch: 8
Time taken for epoch 8 is 2820.6662123203278 sec

--------------------------------------------------------------------------------
ghost_efficientnet_v2_s 	 Epoch  9
train 	 loss: 1.2261584 	 metrics: 0.6616741 	 lr: 0.0003162278
val 	 loss: 1.040364 	 metrics: 0.71245724 	 current_best_epoch: 9
Time taken for epoch 9 is 2819.1916043758392 sec

-----------------------------

KeyboardInterrupt: ignored